<a href="https://colab.research.google.com/github/gorogoro-uk/NLP-project/blob/master/Sherlock_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sherlock HOlmes Text prediction project

In [ ]:
# mount google drive with data
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# imports
import os
from pathlib import Path
import string
nltk.download('punkt')

In [ ]:
# define working directory
os.chdir('./drive/My Drive/sherlock')
print(f"listdir:    {os.listdir(os.getcwd())}")
STORY_FILE = './sh_short.txt'
print(f"STORY_FILE: {STORY_FILE}")

listdir:    ['sh_short.txt']
STORY_FILE: ./sh_short.txt


In [74]:
# Import text story file
with open(STORY_FILE) as reader:
    raw_data = reader.read()   # raw_data is very long string

print(f"raw_data type:    {type(raw_data)}")
print(f"raw_data length:  {len(raw_data)}")
print(raw_data[:300])

raw_data type:    <class 'str'>
raw_data length:  46514

I. A SCANDAL IN BOHEMIA.
I.
To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particular


In [81]:
# remove chapter numbers [I. II. III. etc]
tx1 = raw_data.replace('II.','')
tx1 = tx1.replace('III.','')
tx1 = tx1.replace('IV.','')
tx1 = tx1.replace('V.','')
tx1 = tx1.replace('VI.','')
tx1 = tx1.replace('VII.','')
tx1 = tx1.replace('VIII.','')
tx1 = tx1.replace('IX.','')
tx1 = tx1.replace(' X.',' ')
tx1 = tx1.replace('XI.','')
tx1 = tx1.replace('XII.','')
tx1 = tx1.replace('\nI.','\n')
print(f"tx1:\n{tx1[:300]}")

# replace end of line with space: out=long string
# keep sentence ending punctuation [.?!] to allow sentence splitting below
# include other angled quote marks found in the text [”“‘’]
print(f"punctuation:  {string.punctuation}")
punc_string = "\"#$%&'()*+,-/:;<=>@[\]^_`{|}~'”“‘’—"
print(f"punc string:  {punc_string}")
tt = tx1.maketrans('', '', punc_string)
tx2 = tx1.translate(tt)
tx2 = tx2.replace('\n',' ')
print(f"tx2:\n{tx1[:300]}")

# split into sentences: out=list of strings, each one a sentence
tx3 = nltk.tokenize.sent_tokenize(tx2)
print(f"tx3:\n{tx3[:300]}")
print(f"type:    {type(tx3)}")
print(f"length:  {len(tx3)}")

# remove sentence ending punctuation [.?!]
# make lowercase
# remove white space at start/end
tx4 = [s.translate(s.maketrans('','','.?!')) for s in tx3]
tx4 = [s.lower() for s in tx4]
tx4 = [s.strip() for s in tx4]
print(f"tx4:\n{tx4[:300]}")

tx1:

 A SCANDAL IN BOHEMIA.

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,

punctuation:  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
punc string:  "#$%&'()*+,-/:;<=>@[\]^_`{|}~'”“‘’—
tx2:

 A SCANDAL IN BOHEMIA.

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,

tx3:
['  A SCANDAL IN BOHEMIA.', 'To Sherlock Holmes she is always the woman.', 'I have seldom heard him mention her under any other name.', 'In his eyes she eclipses and predominates the whole of her sex.', 'It was not that he felt any emotion akin to love for Irene Adler.', 'All emotions 